classification_warmup

Classification
Load the swiss data set with pydataset.data
Transform the Catholic variable into a categorical variable named is_catholic. The values should be either Catholic or Not Catholic.
Drop the Catholic column.
Split the data into training and test data sets. We will be trying to predict whether or not a province is catholic.
Fit a decision tree classifier using the Education and Fertility features. Measure the model's performance using accuracy, precision, and recall.
Fit a logistic regression model using Agriculture and Examination. Measure the model's performance.
Fit a K Nearest Neighbors model using two features of your choice. Measure the model's performance.
Use the best model from the ones above on your test data set and evaluate the model's predictions.
Explain how/why your model is making the predictions that it is.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from prepare_utils import get_numeric_columns
from prepare_utils import create_clusters
from prepare_utils import compare_ttests
from prepare_utils import filter_columns
from prepare_utils import regressiontest


%matplotlib inline
from matplotlib import pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn import tree

import statsmodels.api as sm

import scipy.stats as stats
from scipy.stats import pearsonr

import graphviz
from graphviz import Graph

seed = 123

from pydataset import data

%matplotlib notebook

In [2]:
swiss = data('swiss')

In [8]:
swiss.shape

(47, 6)

In [7]:
swiss.describe()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
count,47.000000,47.000000,47.000000,47.000000,47.00000,47.000000
mean,70.142553,50.659574,16.489362,10.978723,41.14383,19.942553
std,12.491697,22.711218,7.977883,9.615407,41.70485,2.912697
min,35.000000,1.200000,3.000000,1.000000,2.15000,10.800000
25%,64.700000,35.900000,12.000000,6.000000,5.19500,18.150000
50%,70.400000,54.100000,16.000000,8.000000,15.14000,20.000000
75%,78.450000,67.650000,22.000000,12.000000,93.12500,21.700000
max,92.500000,89.700000,37.000000,53.000000,100.00000,26.600000


In [12]:
swiss['is_catholic'] = swiss.Catholic > 50


In [13]:
swiss.sample(10)

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality,is_catholic
Rolle,60.5,60.8,16,10,7.72,16.3,False
Le Locle,72.7,16.7,22,13,11.22,18.9,False
Sarine,82.9,45.2,16,13,91.38,24.4,True
Aubonne,66.9,67.5,14,7,2.27,19.1,False
Porrentruy,76.1,35.3,9,7,90.57,26.6,True
Herens,77.3,89.7,5,2,100.00,18.3,True
Cossonay,61.7,69.3,22,5,2.82,18.7,False
Monthey,79.4,64.9,7,3,98.22,20.2,True
Sion,79.3,63.1,13,13,96.83,18.1,True
V. De Geneve,35.0,1.2,37,53,42.34,18.0,False


In [16]:
swiss.drop('Catholic', axis=1)

,Fertility,Agriculture,Examination,Education,Infant.Mortality,is_catholic
Courtelary,80.2,17.0,15,12,22.2,False
Delemont,83.1,45.1,6,9,22.2,True
Franches-Mnt,92.5,39.7,5,5,20.2,True
Moutier,85.8,36.5,12,7,20.3,False
Neuveville,76.9,43.5,17,15,20.6,False
Porrentruy,76.1,35.3,9,7,26.6,True
Broye,83.8,70.2,16,7,23.6,True
Glane,92.4,67.8,14,8,24.9,True
Gruyere,82.4,53.3,12,7,21.0,True
Sarine,82.9,45.2,16,13,24.4,True


In [14]:
def analyze_classification_model(X_df_train, X_df_test, y_df_train, y_df_test, string_criterion, max_depth_input):   
    features = list(X_df_train)
    
    print('Results using ' + str(string_criterion) + ' as the measure of impurity and ' + str(max_depth_input) + ' as the depth.')
    
    print('The features being used: ' + str(features))
    print('-----')

    clf = DecisionTreeClassifier(criterion=string_criterion, max_depth=max_depth_input, random_state=123)

    clf.fit(X_df_train, y_df_train)

    y_df_pred = clf.predict(X_df_train)
    print('Head of predicted on X_train:')
    print(y_df_pred[0:5])
    print('-----')

    y_df_pred_proba = clf.predict_proba(X_df_train)
    print('Head of probabilities on X_train:')
    print(y_df_pred_proba[0:5])
    print('-----')
    
    cm = pd.DataFrame(confusion_matrix(y_df_train, y_df_pred),
             columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])
    print(cm)
    print('-----')
    
    print(classification_report(y_df_train, y_df_pred, digits=4))
    print('-----')

    print('Accuracy of Decision Tree classifier on training set: {:.8f}'.format(clf.score(X_df_train, y_df_train)))
    print('-----')

In [23]:
features = ['Fertility','Education']
X = swiss.drop(['is_catholic'], axis=1)
y = swiss['is_catholic']

In [24]:
train, test = train_test_split(swiss, train_size=0.7, random_state=123)
x_train = train.drop(['is_catholic'], axis=1)
y_train = train.drop(['is_catholic'], axis=1) 

In [ ]:
analyze_classification_model(x_train, X_test, y_train, y_test, 'gini', 5) 